In [12]:
import pandas as pd
import numpy as np
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [13]:
def load_data(file_path):
    """Load the DataFrame from a CSV file."""
    logging.info(f"Loading data from {file_path}")
    return pd.read_csv(file_path)

def check_nulls(df):
    """Check for null values in the DataFrame and log them."""
    null_counts = df.isnull().sum()
    logging.info("Null value counts in the DataFrame:")
    logging.info(null_counts)
    return null_counts


In [14]:
def convert_star_radius(df):
    """Convert Star Radius from solar radii to AU."""
    df['Star Radius (AU)'] = df['Star Radius'] * 0.00484813
    logging.info("Converted Star Radius to AU.")
    return df


In [15]:
def handle_invalid_semi_major_axis(df):
    """Log rows with invalid Semi-Major Axis values."""
    invalid_rows = df[df['Semi-Major Axis (AU)'] <= 0]
    if not invalid_rows.empty:
        logging.warning("Rows with invalid Semi-Major Axis (<= 0):")
        logging.warning(invalid_rows[['Planet Name', 'Semi-Major Axis (AU)']])


In [16]:
def calculate_insolation_flux(df):
    """Calculate Insolation Flux using vectorized operations."""
    valid_entries = (
        df['Star Luminosity'].notnull() &
        df['Distance'].notnull() &
        (df['Distance'] > 0) &
        (df['Star Luminosity'] > 0)
    )

    df.loc[valid_entries, 'Insolation Flux'] = (
        (df.loc[valid_entries, 'Star Luminosity'] * 3.828e26) / 
        (4 * np.pi * (df.loc[valid_entries, 'Distance'] * 3.086e16) ** 2)
    )
    logging.info("Calculated Insolation Flux for valid entries.")
    
    if (~valid_entries).any():
        logging.warning("Rows where Insolation Flux remains null:")
        logging.warning(df[~valid_entries][['Planet Name', 'Star Luminosity', 'Distance', 'Insolation Flux']])

    return df


In [17]:
def calculate_transit_depth(df):
    """Calculate Transit Depth using Planet Radius and Star Radius."""
    valid_entries = (
        df['Planet Radius (Earth Radii)'].notnull() &
        df['Star Radius'].notnull() &
        (df['Star Radius'] > 0)
    )

    df.loc[valid_entries, 'Transit Depth'] = (
        (df.loc[valid_entries, 'Planet Radius (Earth Radii)'] / df.loc[valid_entries, 'Star Radius']) ** 2
    )
    logging.info("Calculated Transit Depth for valid entries.")
    return df


In [18]:
def calculate_star_age(df):
    """Estimate Star Age based on Star Mass."""
    valid_entries = df['Star Mass'].notnull() & (df['Star Mass'] > 0)
    df.loc[valid_entries, 'Star Age (Gyr)'] = 10 * (df.loc[valid_entries, 'Star Mass'] ** -2.5)
    logging.info("Calculated Star Age for valid entries.")
    return df


In [19]:
def calculate_orbital_period(df):
    """Calculate Orbital Period."""
    valid_entries = df['Semi-Major Axis (AU)'].notnull() & df['Star Mass'].notnull() & (df['Star Mass'] > 0)
    df.loc[valid_entries, 'Orbital Period'] = 2 * np.pi * np.sqrt((df.loc[valid_entries, 'Semi-Major Axis (AU)'] ** 3) / df.loc[valid_entries, 'Star Mass'])
    logging.info("Calculated Orbital Period for valid entries.")
    return df


In [20]:
def calculate_equilibrium_temperature(df):
    """Calculate Equilibrium Temperature using vectorized operations."""
    valid_entries = (
        df['Effective Temperature'].notnull() &
        df['Star Radius (AU)'].notnull() &
        df['Semi-Major Axis (AU)'].notnull() &
        (df['Semi-Major Axis (AU)'] > 0)
    )

    df.loc[valid_entries, 'Equilibrium Temperature'] = (
        df.loc[valid_entries, 'Effective Temperature'] *
        (df.loc[valid_entries, 'Star Radius (AU)'] / 
         (2 * df.loc[valid_entries, 'Semi-Major Axis (AU)'])) ** 0.5
    )
    logging.info("Calculated Equilibrium Temperature for valid entries.")
    return df


In [21]:
def save_data(df, file_path):
    """Save the DataFrame to a CSV file."""
    df.to_csv(file_path, index=False)
    logging.info(f"Data saved to {file_path}")


In [22]:
def main():
    # Load the DataFrame from the updated file
    exoplanet_data = load_data('combined_needs_calc.csv')

    # Check for null values
    check_nulls(exoplanet_data)

    # Convert Star Radius
    exoplanet_data = convert_star_radius(exoplanet_data)

    # Handle invalid Semi-Major Axis
    handle_invalid_semi_major_axis(exoplanet_data)

    # Calculate Insolation Flux
    exoplanet_data = calculate_insolation_flux(exoplanet_data)

    # Calculate Transit Depth
    exoplanet_data = calculate_transit_depth(exoplanet_data)

    # Calculate Star Age
    exoplanet_data = calculate_star_age(exoplanet_data)

    # Calculate Orbital Period
    exoplanet_data = calculate_orbital_period(exoplanet_data)

    # Calculate Equilibrium Temperature
    exoplanet_data = calculate_equilibrium_temperature(exoplanet_data)

    # Check for null values after calculations
    check_nulls(exoplanet_data)

    # Display the updated DataFrame to see changes
    logging.info("Relevant columns after calculation:")
    logging.info(exoplanet_data[['Planet Name', 'Insolation Flux', 'Transit Depth', 'Star Age (Gyr)', 'Equilibrium Temperature']].head(10))

    # Save the updated DataFrame to a new CSV file
    save_data(exoplanet_data, 'Calculated_Nulls.csv')

    # Reload to check nulls again from the saved file
    reloaded_data = load_data('Calculated_Nulls.csv')
    check_nulls(reloaded_data)

if __name__ == "__main__":
    main()


2024-10-26 18:49:14,999 - INFO - Loading data from combined_needs_calc.csv
2024-10-26 18:49:15,054 - INFO - Null value counts in the DataFrame:
2024-10-26 18:49:15,055 - INFO - Unnamed: 0                                 0
Planet Name                                0
Planet Letter                              0
Host Star Name                             0
Orbital Period                           281
Orbital Eccentricity                     837
Orbital Inclination                     1416
Planet Radius (Earth Radii)               21
Planet Density                           104
Equilibrium Temperature                 1493
Insolation Flux                         1750
Transit Depth                           1627
Transit Duration                        1528
Star Mass                                  7
Effective Temperature                    244
Star Luminosity                          260
Star Radius                              266
Star Age                                1247
Metallicity  